In [6]:
# import necessary libraries
import numpy as np
import pandas as pd
import scipy.stats as stats
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
from pandas_profiling import ProfileReport
pd.options.display.max_columns = None

In [7]:
# read cleaned Boston data
dataset = ['Boston_marathon_result_2015_cl.xlsx', 'Boston_marathon_result_2016_cl.xlsx', 'Boston_marathon_result_2017_cl.xlsx']
dfs = []
for i in range(0, len(dataset)):
    df = pd.read_excel(dataset[i], index_col=None, header=0)
    df['Year'] = int(''.join(filter(str.isdigit, dataset[i])))
    dfs.append(df)
dataframe = pd.concat(dfs, axis=0, sort=False, ignore_index=True)
dataframe = dataframe.rename(columns={'Gender': 'GRank', 'M/F': 'Gender'})
del dataframe['Unnamed: 2']

In [8]:
# # Summarize dataset
# dataframe.head()
# dataframe.describe()
# # create profile
# profile = ProfileReport(dataframe)
# # save the report
# profile.to_file('boston_profile.html')

In [9]:
# convert string timeseries into floats (minutes)
dataframe['5K'] = pd.to_timedelta(pd.Series(dataframe['5K'].astype(str))) / pd.offsets.Minute(1)
dataframe['10K'] = pd.to_timedelta(pd.Series(dataframe['10K'].astype(str))) / pd.offsets.Minute(1)
dataframe['15K'] = pd.to_timedelta(pd.Series(dataframe['15K'].astype(str))) / pd.offsets.Minute(1)
dataframe['20K'] = pd.to_timedelta(pd.Series(dataframe['20K'].astype(str))) / pd.offsets.Minute(1)
dataframe['Half'] = pd.to_timedelta(pd.Series(dataframe['Half'].astype(str))) / pd.offsets.Minute(1)
dataframe['25K'] = pd.to_timedelta(pd.Series(dataframe['25K'].astype(str))) / pd.offsets.Minute(1)
dataframe['30K'] = pd.to_timedelta(pd.Series(dataframe['30K'].astype(str))) / pd.offsets.Minute(1)
dataframe['35K'] = pd.to_timedelta(pd.Series(dataframe['35K'].astype(str))) / pd.offsets.Minute(1)
dataframe['40K'] = pd.to_timedelta(pd.Series(dataframe['40K'].astype(str))) / pd.offsets.Minute(1)
dataframe['Pace'] = pd.to_timedelta(pd.Series(dataframe['Pace'].astype(str))) / pd.offsets.Minute(1)
dataframe['Official Time'] = pd.to_timedelta(pd.Series(dataframe['Official Time'].astype(str))) / pd.offsets.Minute(1)

In [10]:
# statistical (t-test) function
def statcheck(dataframe1, dataframe2):
    t, p = stats.ttest_ind(dataframe1,dataframe2)
    alpha = 0.05
    if p <= alpha:
        print('Variables are dependent (reject H0)')
    else:
        print('Variables Half2 and Gender are independent (fail to reject H0)')

In [11]:
# estimate function
def estimate(title, dataframe):
    print("{}: mean = {:6.4f}; median = {:6.4f}, std = {:6.4f}".format(title, dataframe.mean(), dataframe.median(), dataframe.std(ddof=1)))

In [12]:
# plotting function
def visualize(dataframe1, dataframe2, title, label1, label2, xlabel, ylabel, box):
    fig, ax = plt.subplots(figsize=(15, 5))
    ax.hist(dataframe1, bins=300, alpha=0.5, color='yellow', label=label1)
    ax.hist(dataframe2, bins=300, alpha=0.2, color='green', label=label2)
    # Set axis labels
    ax.set_xlabel(xlabel, labelpad=30, weight='bold', size=12)
    ax.set_ylabel(ylabel, labelpad=50, weight='bold', size=12)
    # Format y-axis label
    ax.yaxis.set_major_formatter(StrMethodFormatter('{x:,g}'))
    plt.tick_params(axis='x', rotation=0, labelsize=12)
    plt.tick_params(axis='y', rotation=0, labelsize=12)
    plt.axis(box)
    plt.legend(prop={'size': 16})
    plt.suptitle(title, fontsize=16)
    plt.show()

In [ ]:
# Assumption 1a: female runners have a better 2nd half than male runners
dataframe['Half2'] = dataframe['Official Time'] - dataframe['Half']
# In our further assumptions we are interested in those who manage to reach the finish
dataframe = dataframe[dataframe.Half2 > 0]
dataframe['Halfdiff'] = dataframe['Half2'] - dataframe['Half']
for year in dataframe.Year.unique():
    print('Year {0:d}: '.format(year))
    halfdiff_male = dataframe['Halfdiff'][dataframe.Year == year][dataframe.Gender == 'M']
    halfdiff_female = dataframe['Halfdiff'][dataframe.Year == year][dataframe.Gender == 'F']
    statcheck(halfdiff_male, halfdiff_female)
    # estimate means
    estimate('Half difference males', halfdiff_male)
    estimate('Half difference females', halfdiff_female)
    # plot histograms
    visualize(halfdiff_male, halfdiff_female, "First / second half marathon time difference by gender in year {0:d}".format(year), 'Males', 'Females', 'Half difference (Minutes)', 'Count', [-25, 100, 0, 1000])

In [ ]:
# Assumption 1b: female runners have a better last 10k than male runners
dataframe['last10K'] = dataframe['Official Time'] - dataframe['30K']
dataframe['10Kdiff'] = dataframe['last10K'] - dataframe['10K']
for year in dataframe.Year.unique():
    print('Year {0:d}: '.format(year))
    fl10Kdiff_male = dataframe['10Kdiff'][dataframe.Year == year][dataframe.Gender == 'M']
    fl10Kdiff_female = dataframe['10Kdiff'][dataframe.Year == year][dataframe.Gender == 'F']
    statcheck(fl10Kdiff_male, fl10Kdiff_female)
    # estimate means
    estimate('First-last 10K difference males', fl10Kdiff_male)
    estimate('First-last 10K difference females', fl10Kdiff_female)
    # plot histograms
    visualize(fl10Kdiff_male, fl10Kdiff_female, "First-last 10 kilometers time difference by gender in year {0:d}".format(year), 'Males', 'Females', '10K difference (Minutes)', 'Count', [-25, 100, 0, 1000])

In [ ]:
# Assumption 2: male runners have a faster 1st half than female runners
for year in dataframe.Year.unique():
    print('Year {0:d}: '.format(year))
    dataframe['Halfratio'] = dataframe['Half'] / dataframe['Half2']
    halfratio_male = dataframe['Halfratio'][dataframe.Year == year][dataframe.Gender == 'M']
    halfratio_female = dataframe['Halfratio'][dataframe.Year == year][dataframe.Gender == 'F']
    statcheck(halfratio_male,halfratio_female)
    # estimate means
    estimate('Half-ratio males', halfratio_male)
    estimate('Half-ratio females', halfratio_female)
    # plot histograms
    visualize(halfratio_male, halfratio_female, "Distribution of the 1st / 2nd half-ratio by gender in year {0:d}".format(year), 'Males', 'Females', 'Half-ratio', 'Count', [0, 5, 0, 400])

In [ ]:
# Assumption 5: local runners (from Boston or close) have a more stable run than non locals
Neighbours = ["Boston", "Winthrop", "Revere", "Chelsea", "Everett", "Somerville", "Cambridge", "Watertown", "Newton", "Brookline", "Needham", "Dedham", "Canton", "Milton", "Quincy"]
for year in dataframe.Year.unique():
    print('Year {0:d}: '.format(year))
    local = dataframe['Pace'][dataframe.Year == year][dataframe.City.isin(Neighbours)]
    nolocal = dataframe['Pace'][dataframe.Year == year][~dataframe.City.isin(Neighbours)]
    statcheck(local, nolocal)
    # estimate means
    estimate('Pace Locals', local)
    estimate('Pace non-Locals', nolocal)
    print("Numbers: locals = {}; non-locals = {}".format(len(local), len(nolocal)))
    # plot histograms
    visualize(local, nolocal, "Distribution of the marathon pace by locality in year {0:d}".format(year), 'Locals', 'Non-locals', 'Pace (Minutes)', 'Count', [0, 25, 0, 500])

In [ ]:
# Assumption 6: runners who return do better (more stable and maybe faster, lower suffer score)
Returns = dataframe[dataframe.Name.duplicated()]['Name'].tolist()
for year in dataframe.Year.unique():
    print('Year {0:d}: '.format(year))
    ret = dataframe['Pace'][dataframe.Year == year][dataframe.Name.isin(Returns)]
    nonret = dataframe['Pace'][dataframe.Year == year][~dataframe.Name.isin(Returns)]
    statcheck(ret, nonret)
    # estimate means
    estimate('Pace Return', ret)
    estimate('Pace Non-Return', nonret)
    print("Numbers: return = {}; non-return = {}".format(len(local), len(nolocal)))
    # plot histograms
    visualize(ret, nonret, "Distribution of the marathon pace by return in year {0:d}".format(year), 'Return', 'Non-Return', 'Pace (Minutes)', 'Count', [0, 25, 0, 500])

In [ ]:
# read Weather data
dataset = 'weather_data_boston.csv'
weather = pd.read_csv(dataset, header=0)
weather.head()

In [ ]:
# Assumption 3: higher levels of humidity / temperature have a negative impact in A1 and A2
# take a sample of returning runners and test average humidity levels on them
Events = ['2015-04-20T10:00:00', '2016-04-18T10:00:00', '2017-04-17T10:00:00']
returns = dataframe[dataframe.Name.isin(Returns)]
returns_A1 = returns['5K']
returns_A2 = returns['10K']
# for event in Events:
#     dt = weather.iloc[(df['DATE']-event).abs().argsort()[:2]] # TODO: convert to floats and get minmax mean